## Training a GP surrogate model

This notebook demonstrates how to train a GP model to represent the outputs of an ODE model.


In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display
import pickle

from vpop_calibration import *

### Generate a training data set


In [ ]:
# Define the ode model
def equations(t, y, k_12, k_21, k_el):
    ydot = [  # y[0] is A1, y[1] is A2
        k_21 * y[1] - k_12 * y[0] - k_el * y[0],
        k_12 * y[0] - k_21 * y[1],
    ]
    return ydot


variable_names = ["A0", "A1"]
parameter_names = ["k_12", "k_21", "k_el"]

pk_two_compartments_model = OdeModel(equations, variable_names, parameter_names)

In [ ]:
nb_timesteps = 15
tmax = 24.0
initial_conditions = np.array([10.0, 0.0])
time_steps = np.linspace(0.0, tmax, nb_timesteps)

log_nb_patients = 5
nb_patients = 2**log_nb_patients
param_ranges = {
    "k_12": {"low": 0.02, "high": 0.07, "log": False},
    "k_21": {"low": 0.1, "high": 0.3, "log": False},
}

protocol_design = pd.DataFrame({"protocol_arm": ["A", "B"], "k_el": [0.1, 0.5]})

In [ ]:
# Generate a noise-free data set
dataset = simulate_dataset_from_ranges(
    pk_two_compartments_model,
    log_nb_patients,
    param_ranges,
    initial_conditions,
    protocol_design,
    None,
    None,
    time_steps,
)

In [ ]:
# Remove parts of the data set
bootstrapped_dataset = dataset.sample(frac=0.5)

### Define and train a GP model


In [ ]:
learned_ode_params = list(param_ranges.keys())
descriptors = learned_ode_params + ["time"]
print(descriptors)

# initiate our GP class
myGP = GP(
    bootstrapped_dataset,
    descriptors,
    var_strat="IMV",  # either IMV (Independent Multitask Variational) or LMCV (Linear Model of Coregionalization Variational)
    kernel="RBF",  # Either RBF or SMK
    nb_inducing_points=100,
    mll="ELBO",  # default, otherwise PLL
    nb_training_iter=200,
    training_proportion=0.7,
    learning_rate=0.1,
    lr_decay=0.99,
    jitter=1e-6,
    log_inputs=learned_ode_params,
    log_outputs=["A0", "A1"],
)

In [ ]:
myGP.train()

### Assess the goodness of fit of the GP


In [ ]:
myGP.plot_loss()
myGP.eval_perf()

In [ ]:
myGP.plot_obs_vs_predicted(data_set="training")

In [ ]:
j = int(np.random.randint(0, nb_patients))
myGP.plot_individual_solution(j)

In [ ]:
myGP.plot_all_solutions("training")

## Save your trained model in a pickle for later use


In [ ]:
model_file = "gp_model_example.pkl"
folder_path = "./"
with open(folder_path + model_file, "wb") as file:
    pickle.dump(myGP, file)
print(f"Model saved to {model_file}")